In [2]:
#%pip install dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [8]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years
year_list = [i for i in range(1980, 2024, 1)]

#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center',
        'color': '#503D36', 'font-size': 24})
    ,
    #May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Select Statistics',
            placeholder='Select a Report Type'
        )
    ]),
    html.Div(dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value='Select Year',
            placeholder='Select the Year'
        )),
    html.Div([#TASK 2.3: Add a division for output display
              html.Div(id='output-container1', className='chart-item', style={'display': 'flex'}),
              html.Div(id='output-container2', className='chart-item', style={'display': 'flex'})
              ])
])

#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    [Output(component_id='select-year', component_property='disabled'), Output(component_id='select-year', component_property='value')],
    Input(component_id='dropdown-statistics',component_property='value'))

def update_input_container(input):
    if input =='Yearly Statistics':
        return [False,'']
    else:
        return [True,'']

#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    [Output(component_id='output-container1', component_property='children'), Output(component_id='output-container2', component_property='children')],
    [Input(component_id='dropdown-statistics', component_property='value'), Input(component_id='select-year', component_property='value')])

def update_output_container(report, input_year):
    ### Yearly Statistic Report Plots
    if(report=='Yearly Statistics'):
        # Filter the data for selected year
        yearly_data = data[data['Year'] == input_year]
        g1, g2, g3, g4 = getGraphYearly(yearly_data, input_year)

        #TASK 2.5: Returning the graphs for display
        return displayGraphs(g1, g2, g3, g4)

    ### Recession period report Plots
    elif(report=='Recession Period Statistics'):
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        g1, g2, g3, g4 = getGraphRec(recession_data)

        #TASK 2.6: Returning the graphs for display
        return displayGraphs(g1, g2, g3, g4)

def displayGraphs(g1, g2, g3, g4):
  return[
            html.Div(className='chart-item', children=[html.Div(children=g1),html.Div(children=g2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=g3),html.Div(children=g4)], style={'display': 'flex'})
        ]

def getGraphYearly(yearly_data, input_year):
  ## Plot 1: Yearly Automobile sales using line chart for the whole period.
  # grouping data for plotting
  yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
  # Plotting the line graph
  Yearly_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title='Mean Automobile sales for the Whole Period'))

  ## Plot 2 Total Monthly Automobile sales using line chart.
  # grouping data for plotting
  yts= yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
  # Ordering Months for Plotting
  ordered_months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
  yts['Month'] = pd.Categorical(yts['Month'], categories=ordered_months, ordered=True)
  yts['to_sort']=yts['Month'].apply(lambda x:ordered_months.index(x))
  yts = yts.sort_values('to_sort')
  # Plotting the line graph
  Yearly_chart2 = dcc.Graph(figure=px.line(yts, x='Month', y='Automobile_Sales', title='Total Monthly Automobile Sales in the year {}'.format(input_year)))

  # Plot bar chart for average number of vehicles sold during the given year
  # grouping data for plotting
  avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
  # Plotting the Bar graph
  Yearly_chart3 = dcc.Graph(figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

  # Total Advertisement Expenditure for each vehicle using pie chart
  # grouping data for plotting
  exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
  # Plotting the Pie graph
  Yearly_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                                      title="Total Expenditure share by Vehicle Type in the year {}".format(input_year)))

  return [Yearly_chart1, Yearly_chart2, Yearly_chart3, Yearly_chart4]

def getGraphRec(recession_data):
  ## Plot 1: Automobile sales fluctuate over Recession Period (year wise)
  # grouping data for plotting
  yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
  # Plotting the line graph
  Rec_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Automobile sales fluctuation over Recession Period"))

  ## Plot 2: Calculate the average number of vehicles sold by vehicle type
  # grouping data for plotting
  average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
  # Plotting the line graph
  Rec_chart2  = dcc.Graph(figure=px.line(average_sales, x='Vehicle_Type', y='Automobile_Sales', title="Average Vehicles Sold by Vehicle Type during Recession"))

  ## Plot 3: Pie chart for total expenditure share by vehicle type during recessions
  # grouping data for plotting
  exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
  # Plotting the Pie graph
  Rec_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type', title="Total Expenditure by Vehicle Type during Recessions"))

  ## Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
  # grouping data for plotting
  unemp_rec= recession_data.groupby(['Vehicle_Type', 'Automobile_Sales'])['unemployment_rate'].mean().reset_index()
  # Plotting the Bar graph
  Rec_chart4 = dcc.Graph(figure=px.bar(unemp_rec, x='Vehicle_Type', y='unemployment_rate', color='Automobile_Sales',
                                    title='Effect of Unemployment Rate on Vehicle Type and Sales during Recession'))

  return [Rec_chart1, Rec_chart2, Rec_chart3, Rec_chart4]

if __name__ == '__main__':
    app.run_server(port=8050)

<IPython.core.display.Javascript object>